In [11]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd 

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Importation des données

In [12]:
# Instance_file : str
Instance_file = "InstanceBordeauxV1.xlsx"

# Chargement des donnees
Employees = pd.read_excel(Instance_file, sheet_name= 'Employees')
Tasks = pd.read_excel(Instance_file, sheet_name= 'Tasks')
Employees_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Employees Unavailabilities')
Tasks_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Tasks Unavailabilities')

print(Employees)

  EmployeeName   Latitude  Longitude     Skill  Level WorkingStartTime  \
0     Valentin  45.151218  -0.822093  Oenology      2           7:00am   
1        Ambre  45.199575  -0.822093  Oenology      1           8:00am   

  WorkingEndTime  
0         5:00pm  
1         6:00pm  


## Création des dictionnaires de paramètres

Listes des dictionnaires de paramètres à créer:
- LevelEmployee[i] qui à un employé i lui associe son niveau de compétence
- BeginningEmployee[i] qui à un employé i lui associe son début de journée de travail
- EndEmployee[i] qui à un employé i lui associe sa fin de journée de travail
- LevelTask[j] qui à une tâche j lui associe son niveau de difficulté
- BeginningTask[j] qui à une tâche j lui aussi le début du créneau durant lequel on peut l'effectuer
- EndTask[j] qui à une tâche j lui aussi la fin du créneau durant lequel on peut l'effectuer
- DurationTask[j] qui à une tâche j lui associe sa durée
- TasksUnavailabilities[j]
- DistDict[i][j] dictionnaire contenant les distances entre toutes les tâches et les domiciles des employées
- TimeDict[i][j] dictionnaire contenant les temps de trajet associés
- Unavailability[l]: recense les indisponibilités sous forme (employe, debutindispo, duree)

In [46]:
LevelEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["Level"][i]
    LevelEmployee[a] = b

#print(LevelEmployee)

BeginningEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingStartTime"][i][:1]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    if Employees["WorkingStartTime"][i][-2:] == 'am':
        BeginningEmployee[a] = b
    if Employees["WorkingStartTime"][i][2:] == "pm":
        BeginningEmployee[a] = b + 12

#print(BeginningEmployee)

EndEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingEndTime"][i][:1]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60

    if Employees["WorkingEndTime"][i][-2:] == 'am':
        EndEmployee[a] = b
    if Employees["WorkingEndTime"][i][2:] == "pm":
        EndEmployee[a] = b + 12

#print(EndEmployee)

LevelTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = i+1
    b = Tasks["Level"][i]
    LevelTask[a] = b

#print(LevelTask)

BeginningTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = Tasks["TaskId"][i]
    b = Tasks["OpeningTime"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    if Tasks["OpeningTime"][i][-2:] == 'am':
        BeginningTask[a] = b
    if Tasks["OpeningTime"][i][2:] == "pm":
        BeginningTask[a] = b + 12


#print(BeginningTask)


EndTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = Tasks["TaskId"][i] 
    b = Tasks["ClosingTime"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60

    if Tasks["ClosingTime"][i][-2:] == 'am':
        EndTask[a] = b
    if Tasks["ClosingTime"][i][2:] == "pm":
        EndTask[a] = b + 12

#print(EndTask)

DurationTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = i+1
    b = Tasks["TaskDuration"][i] / 60
    DurationTask[a] = b

#print(DurationTask)


TasksUnavailabilities = dict()

for i in range (len(Tasks_Unavailabilities["TaskId"])):
    a = Tasks_Unavailabilities["TaskId"][i] 
    b = Tasks_Unavailabilities["Start"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    c = Tasks_Unavailabilities["End"][i][:5]
    cg = c[:2]   #prend les deux premiers caractères
    cd = c[-3:]
    if cg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        c = int(c[:1]) + int(cd[:2]) *100/60
        
    if cg[-1:] != ":":
        c = int(c[:2]) + int(cd[-2:]) *100/60

    if Tasks_Unavailabilities["Start"][i][-2:] == 'pm' and b!=12:
        b = b + 12
    if Tasks_Unavailabilities["End"][i][2:] == "pm" and c!=12:
        c = c + 12
    TasksUnavailabilities[a] = [b,c]

#print(TasksUnavailabilities)

Unavailability = dict()

for i in range (len(Employees_Unavailabilities["EmployeeName"])):
    a = Employees_Unavailabilities["EmployeeName"][i] 
    b = Employees_Unavailabilities["Start"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    c = Employees_Unavailabilities["End"][i][:5]
    cg = c[:2]   #prend les deux premiers caractères
    cd = c[-3:]
    if cg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        c = int(c[:1]) + int(cd[:2]) *100/60
        
    if cg[-1:] != ":":
        c = int(c[:2]) + int(cd[-2:]) *100/60
    if Employees_Unavailabilities["Start"][i][-2:] == 'pm'and b!=12:
        b = int(b) + 12
    if Employees_Unavailabilities["End"][i][2:] == "pm" and c!=12:
        c = int(c) + 12 - b
    Unavailability[i] = [a,b,c]



#print(Unavailability)


##Création du dictionnaire des distances

DistDict = dict()
R = 6371  #rayon de la Terre

for i in range (len(Tasks["TaskId"])):               ## ajout des distances entre tasks
    for k in range (len(Tasks["TaskId"])):
        if k!=i:
            L_i = Tasks["Longitude"][i]
            l_i = Tasks["Latitude"][i]
            L_k = Tasks["Longitude"][k]
            l_k = Tasks["Latitude"][k]
            A = np.sin((l_i-l_k)*np.pi/(2*180))**2
            B = (np.sin((L_i-L_k)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_k*np.pi/180)
            d = 2*R*np.sqrt(A+B)
            DistDict[(i+1,k)] = d
            DistDict[(k,i+1)] = d

for i in range (len(Tasks["TaskId"])):             #### ajout des disatnces tasks - domiciles des employés
    for k in range (len(Employees["EmployeeName"])):
        L_i = Tasks["Longitude"][i]
        l_i = Tasks["Latitude"][i]
        L_E = Employees["Longitude"][k]
        l_E = Employees["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        DistDict[(i+1, Employees["EmployeeName"][k])] = d 
        DistDict[(Employees["EmployeeName"][k], i+1)] = d       

for i in range (len(Tasks["TaskId"])):
    for k in range (len(Employees_Unavailabilities["EmployeeName"])):
        L_i = Tasks["Longitude"][i]
        l_i = Tasks["Latitude"][i]
        L_E = Employees["Longitude"][k]
        l_E = Employees["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        C = Employees_Unavailabilities["EmployeeName"][k] + "U{}".format(k)   ##############Revoir numerotation des unavailabilities employé pour les retrouver plus vite lors de l'optimisation
        DistDict[(i+1, C)] = d 
        DistDict[(C, i+1)] = d    

#print(DistDict)

##Création dictionnaire des temps de trajet

TimeDict = dict()

for element in DistDict.items():
    t = element[1] / 50
    TimeDict[element[0]] = t

#print(TimeDict)



{0: ['Ambre', 12.0, 1.0]}
